Сделайте несколько усовершенствований в предсказании временного ряда

1. Добавьте к исходному временному ряду следующие данные, в виде новых каналов:

попарные разности каналов
модули попарных разностей каналов
попарные произведения каналов
обратное значение каналов x_new = 1/(x + 1e-3)
первые производные каналов (x[n] - x[n-1])
вторые производные каналов (x[n] - 2*x[n-1] + x[n-2])
* каналы - столбцы датафрейма OPEN, MAX, MIN, CLOSE.

2. Сделайте предсказание нейросети, сравните эталонный сигнал с настоящим. При формировании входного сигнала используйте:

100 точек с шагом назад по 1
100 точек с шагом назад по 10 (или сами точки, или среднее по отрезку в 10 точек)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical 
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import concatenate, Input, Dense, Dropout, BatchNormalization, Flatten, Conv1D, Conv2D, LSTM, GlobalMaxPooling1D, MaxPooling1D, RepeatVector #Стандартные слои
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
base_data16 = pd.read_csv('/content/drive/MyDrive/Lessons/Lesson7/16_17.csv', sep=';')
base_data18 = pd.read_csv('/content/drive/MyDrive/Lessons/Lesson7/18_19.csv', sep=';')
data16_17 = base_data16.iloc[:,2:]
data18_19 = base_data18.iloc[:,2:]

In [ ]:
data16_17.shape[0]

263925

In [ ]:
d = data18_19
print(len(d))
print(len(d.iloc[0]))
print(d.iloc[0])

217947
5
OPEN      3341.0
MAX       3356.0
MIN       3340.0
CLOSE     3355.0
VOLUME    6088.0
Name: 0, dtype: float64


In [ ]:
data = data16_17.copy()
data = pd.concat([data16_17,data18_19])
data = np.array(data)

In [ ]:
open = data[:,0]
max = data[:,1]
min = data[:,2]
close = data[:,3]

In [ ]:

d = list()

# Так задача прогнозировать в обратном порядке
for ii in range(len(open)):
  d.append([open[- 1 - ii], close[- 1 - ii]])

In [ ]:
def get_part_data(data):
  new_data = list()
  data = np.array(data)
  for ii in range(len(data)):
    elem = list()
    elem.append(data[ii,0] - data[ii,1])
    elem.append(abs(data[ii,0] - data[ii,1]))
    elem.append(data[ii,0] * data[ii,1])
    for jj in range(2):
      elem.append(1 / (data[ii,jj] + 1e-3))
    if ii == 0:
      for jj in range(2):
        elem.append(0)
    else:
      for jj in range(2):
        elem.append(data[ii,jj] - data[ii-1,jj])

    if ii < 2:
      for jj in range(2):
        elem.append(0)
    else:
      for jj in range(2):
        elem.append(data[ii,jj] - 2*data[ii-1,jj] + data[ii-2,jj])
    new_data.append(elem)
  return np.array(new_data)


In [ ]:
dt = get_part_data(d)
# data = np.array(data)
print(dt.shape)


(481872, 9)


In [ ]:
print(dt[5])

[-2.00000000e+00  2.00000000e+00  2.67392400e+07  1.93423560e-04
  1.93348764e-04 -1.35000000e+01 -1.15000000e+01 -1.35000000e+01
 -1.15000000e+01]


In [ ]:
xScaler = MinMaxScaler()
xScaler.fit(data)

xx = xScaler.transform(data)

In [ ]:
print(dt[:100].shape)

(100, 9)


In [ ]:
xLen = 100
trainLen = len(dt) - 30000
testLen = len(dt) - trainLen - xLen - xLen
yTrain = np.reshape(dt[xLen:trainLen + xLen], (-1, 9))
yTest = np.reshape(dt[trainLen + xLen * 2:], (-1, 9))

In [ ]:
np.reshape(np.array([[1, 2, 5], [3, 4, 3], [4, 5, 4]]), (-1, 3))
dt[xLen:trainLen + xLen].shape

(451872, 9)

## Получение данных

In [ ]:
def get_data(data, xLen=100, valLen=30000):

  trainLen = len(data) - valLen

  testLen = len(data) - trainLen - xLen*2
 
  xTrain = data[:trainLen].reshape(trainLen, 9)
  xTest = data[trainLen + xLen: -xLen].reshape(testLen, 9)

  yTrain = np.reshape(data[xLen:trainLen + xLen], (-1, 9))
  yTest = np.reshape(data[trainLen + xLen * 2:], (-1, 9))

  xScaler = MinMaxScaler()
  xScaler.fit(xTrain)

  yScaler = MinMaxScaler()
  yScaler.fit(yTrain)

  xTrain = xScaler.transform(xTrain)
  xTest = xScaler.transform(xTest)
  
  yTrain = yScaler.transform(yTrain)
  yTest = yScaler.transform(yTest)

  print(len(xTrain))
  print(len(xTest))
  print(len(yTrain))
  print(len(yTest))
  # 451872
  # 29800
  # 2259360
  # 149000  

  trainDataGen = TimeseriesGenerator(xTrain, yTrain, 
                                length=xLen, stride=1, sampling_rate=1,
                                batch_size=20)

  testDataGen = TimeseriesGenerator(xTest, yTest,
                                length=xLen, stride=1,
                                batch_size=20)
   
  DataGen = TimeseriesGenerator(xTest, yTest,
                               length=xLen,
                               batch_size=len(xTest))
  
  xVal = []
  yVal = []
  for i in DataGen:
    xVal.append(i[0])
    yVal.append(i[1])

  xVal = np.array(xVal)
  yVal = np.array(yVal)
    


  return [trainDataGen, testDataGen, xScaler, yScaler, xVal, yVal]


In [ ]:
print(trainDataGen[0][0].shape,
      trainDataGen[0][1].shape)

(4, 5, 3) (4,)


In [ ]:
def getPred(currModel, xVal, yVal, yScaler):

  predVal = yScaler.inverse_transform(currModel.predict(xVal))
  yValUnscaled = yScaler.inverse_transform(yVal)
  
  return (predVal, yValUnscaled)


def showPredict(start, step, channel, predVal, yValUnscaled):
  plt.figure(figsize=(22,6))
  plt.plot(predVal[start:start+step, 0],
           label='Прогноз')
  plt.plot(yValUnscaled[start:start+step, channel], 
           label='Базовый ряд')
  plt.xlabel('Время')
  plt.ylabel('Значение Close')
  plt.legend()
  plt.show()
  

def correlate(a, b):

  ma = a.mean()
  mb = b.mean()
  mab = (a*b).mean()
  sa = a.std()
  sb = b.std()
  
  val = 1
  if ((sa>0) & (sb>0)):
    val = (mab-ma*mb)/(sa*sb)
  return val


def showCorr(channels, corrSteps, predVal, yValUnscaled):

  for ch in channels:
    corr = []
    yLen = yValUnscaled.shape[0]

    for i in range(corrSteps):

      corr.append(correlate(yValUnscaled[:yLen-i,ch], predVal[i:, 0]))

    own_corr = []

     
    for i in range(corrSteps):
      own_corr.append(correlate(yValUnscaled[:yLen-i,ch], yValUnscaled[i:, ch]))

    plt.figure(figsize=(14,7))
    plt.plot(corr, label='Предсказание на ' + str(ch+1) + ' шаг')
    plt.plot(own_corr, label='Эталон')

  plt.xlabel('Время')
  plt.ylabel('Значение')
  plt.legend()
  plt.show()

In [ ]:
data_for_model = get_data(dt)
trainDataGen, testDataGen = data_for_model[0], data_for_model[1]
xScaler, yScaler = data_for_model[2], data_for_model[3]
xVal, yVal = data_for_model[4], data_for_model[5]

451872
29800
451872
29800


In [ ]:
modelD = Sequential()
modelD.add(Dense(150,input_shape = (xLen,9), activation="relu" ))
modelD.add(Flatten())
modelD.add(Dense(9, activation="linear"))

modelD.compile(loss="mse", optimizer=Adam(lr=1e-4))

modelD.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100, 150)          1500      
                                                                 
 flatten (Flatten)           (None, 15000)             0         
                                                                 
 dense_1 (Dense)             (None, 9)                 135009    
                                                                 
Total params: 136,509
Trainable params: 136,509
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 25

In [ ]:
history = modelD.fit(
                    trainDataGen, 
                    epochs=epochs, 
                    verbose=1, 
                    validation_data = testDataGen 
                    )

Epoch 1/25
22589/22589 [==============================] - 140s 6ms/step - loss: 3.4054e-04 - val_loss: 0.0011
Epoch 2/25
22589/22589 [==============================] - 141s 6ms/step - loss: 1.6072e-04 - val_loss: 0.0046
Epoch 3/25
22589/22589 [==============================] - 139s 6ms/step - loss: 1.4743e-04 - val_loss: 0.0028
Epoch 4/25
22589/22589 [==============================] - 140s 6ms/step - loss: 1.4033e-04 - val_loss: 0.0014
Epoch 5/25
22589/22589 [==============================] - 147s 6ms/step - loss: 1.3736e-04 - val_loss: 0.0018
Epoch 6/25
22589/22589 [==============================] - 148s 7ms/step - loss: 1.3563e-04 - val_loss: 0.0025
Epoch 7/25
22589/22589 [==============================] - 147s 7ms/step - loss: 1.3426e-04 - val_loss: 7.6219e-04
Epoch 8/25
22589/22589 [==============================] - 145s 6ms/step - loss: 1.3414e-04 - val_loss: 0.0030
Epoch 9/25
22589/22589 [==============================] - 148s 7ms/step - loss: 1.3217e-04 - val_loss: 0.0017
Epoch 